# 불법 현수막 탐지 프로젝트 Main

### <시나리오>

* 촬영 데이터 입력
* Yolov8 현수막 추출
* OCR 텍스트 추출
* chatGPT 대분류
* Dataframe에 기록

##### 1) 마을버스/순찰차/순찰 시 GPS 기준 이동거리 5m 간격 촬영

In [1]:
import json

# 촬영된 데이터 format 예시
# 16개의 사진 데이터 
json_file_path = 'capture_data/meta_data.json'

with open(json_file_path, 'r') as json_file:
    meta_data = json.load(json_file)

# 카메라 촬영을 통해 필요한 데이터 요소
# id(고유키), filename(파일명), timestamp(날짜/시간), location(위도/경도)
meta_data

{'dataset_info': {'name': '불법 현수막 탐지 프로젝트 데이터셋',
  'version': '1.0',
  'description': '도로에서 촬영한 불법 현수막 탐지 데이터셋',
  'date_created': '2023-08-12T15:00:00.123456'},
 'data': [{'id': 1,
   'file_name': 'data_001.jpg',
   'timestamp': '2023-08-12T15:00:00.123456',
   'location': {'latitude': 37.123456, 'longitude': -122.789012}},
  {'id': 2,
   'file_name': 'data_002.jpg',
   'timestamp': '2023-08-12T15:05:00.123456',
   'location': {'latitude': 37.123501, 'longitude': -122.789012}},
  {'id': 3,
   'file_name': 'data_003.jpg',
   'timestamp': '2023-08-12T15:10:00.123456',
   'location': {'latitude': 37.123546, 'longitude': -122.789012}},
  {'id': 4,
   'file_name': 'data_004.jpg',
   'timestamp': '2023-08-12T15:15:00.123456',
   'location': {'latitude': 37.123591, 'longitude': -122.789012}},
  {'id': 5,
   'file_name': 'data_005.jpg',
   'timestamp': '2023-08-12T15:20:00.123456',
   'location': {'latitude': 37.123636, 'longitude': -122.789012}},
  {'id': 6,
   'file_name': 'data_006.jpg',
 

##### 2) yolov8 활용 데이터별(id별) 현수막 crop

In [2]:
# 학습한 모델 load

# https://stackoverflow.com/questions/24773109/how-to-downgrade-the-installed-version-of-pip-on-windows
# pip install torchvision==0.15.1
from ultralytics import YOLO
import torch
from datetime import datetime
import shutil
import os
from glob import glob

# cpu
# device = 'cpu'

# gpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

model = YOLO('models/08_10_banner576_best.pt')

cuda


In [4]:
import pandas as pd
from PIL import Image
import numpy as np

# DataFrame 정의

# <Columns>  
# Id(고유 키)
# Date(날짜) 
# Time(시간) 
# Location(위치) 
# Origin_img(원본사진) 
# Crop_imgs(detect된 사진들) 
# OCR_text(추출 텍스트)
# Category(현수막 종류) 
# Legality(합/불법)  


print(" meta data 요소 : ",meta_data['data'][0].keys())

df_data = []
for item in meta_data['data']:

    date_time = item['timestamp'].split("T")
    df_row = {
        'Id' : item['id'],
        'Date' : date_time[0],
        'Time' : date_time[1],
        'Location' : [item['location']['latitude'], item['location']['longitude']],
        'Origin_img' : np.array(Image.open('capture_data/'+item['file_name'])),
        'Detect_img' : [],
        'Crop_imgs' : [],
        'OCR_text' : [],
        'Category' : [],
        'Legality' : []
    }

    df_data.append(df_row)

df_report = pd.DataFrame(df_data)

# date_created : 데이터 수집한 날짜
date_created = meta_data['dataset_info']['date_created'].split("T")[0]
df_report.to_csv('reports/report_'+date_created+'.csv')
df_report

 meta data 요소 :  dict_keys(['id', 'file_name', 'timestamp', 'location'])


,Id,Date,Time,Location,Origin_img,Detect_img,Crop_imgs,OCR_text,Category,Legality
0,1,2023-08-12,15:00:00.123456,"[37.123456, -122.789012]","[[[76, 104, 105], [74, 102, 103], [84, 112, 11...",[],[],[],[],[]
1,2,2023-08-12,15:05:00.123456,"[37.123501, -122.789012]","[[[38, 37, 35], [46, 45, 43], [73, 73, 71], [8...",[],[],[],[],[]
2,3,2023-08-12,15:10:00.123456,"[37.123546, -122.789012]","[[[21, 44, 24], [5, 28, 8], [9, 31, 10], [14, ...",[],[],[],[],[]
3,4,2023-08-12,15:15:00.123456,"[37.123591, -122.789012]","[[[60, 102, 127], [101, 143, 168], [111, 153, ...",[],[],[],[],[]
4,5,2023-08-12,15:20:00.123456,"[37.123636, -122.789012]","[[[175, 205, 205], [169, 191, 188], [79, 92, 8...",[],[],[],[],[]
5,6,2023-08-12,15:25:00.123456,"[37.123681, -122.789012]","[[[95, 107, 93], [101, 111, 100], [108, 118, 1...",[],[],[],[],[]
6,7,2023-08-12,15:30:00.123456,"[37.123726, -122.789012]","[[[113, 113, 105], [97, 97, 89], [86, 86, 78],...",[],[],[],[],[]
7,8,2023-08-12,15:35:00.123456,"[37.123771, -122.789012]","[[[26, 31, 37], [28, 33, 39], [32, 37, 41], [4...",[],[],[],[],[]
8,9,2023-08-12,15:40:00.123456,"[37.123816, -122.789012]","[[[251, 248, 243], [249, 248, 243], [248, 251,...",[],[],[],[],[]
9,10,2023-08-12,15:45:00.123456,"[37.123861, -122.789012]","[[[123, 159, 89], [109, 144, 80], [175, 208, 1...",[],[],[],[],[]


In [36]:
class_names = ['banner','frame']

predict_save_path = 'runs/detect/predict/'
predict_log_path = 'predict_logs/'

croped_banner_save_path = predict_save_path+ 'crops/'
croped_banner_log_path = predict_log_path+ 'crops/'

image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif']

cnt_predict = 1

def run(img):
    global cnt_predict
    
    results = model.predict(
                            source=img, # 이미지 경로
                             conf=0.5, # confidence threshold for detection (오탐지 시 재설정)
                            save=True,  # Detect 결과 저장 (runs/detect/predict)
                            device=device, # device 설정
                            show=False, # window 창으로 show
                            save_crop=True # Detect된 Obeject 사진 저장 (runs/detect/predict/crops)
                            )

    for result in results:
        boxes = result.boxes  
        print("================================= Predict 결과 =================================")

        saved_img = ''

        for extension in image_extensions:
            try:
                saved_img = glob(os.path.join(predict_save_path, extension))[0]
            except:
                pass
        print(saved_img)

        now = datetime.now()
        now_time = str(now.year) + str(now.month) + str(now.day) + '_' + str(now.hour) + str(now.minute) + str(now.second)
        # predict_log_path = 'predict_logs/'

        img_name = saved_img.split('\\')[-1][:-4]
        ext_name = saved_img.split('\\')[-1][-4:]

        # 원본 이미지 저장
        shutil.copyfile(img,predict_log_path+'origin/'+str(cnt_predict)+'_'+img_name+'_'+now_time+ext_name) 

        # boxing 된 이미지 저장
        shutil.copyfile(saved_img,predict_log_path+'predict/boxes/'+str(cnt_predict)+'_'+img_name+'_'+now_time+ext_name) 

        cnt_predict += 1

        # crop_imgs = gl 

        for idx,box in enumerate(boxes):
            
            # crop 된 이미지 저장
            # crop 되면 전부 jpg로 출력

            # shutil.copyfile(croped_banner_save_path+'*jpg',predict_log_path+'predict/boxes/'+str(cnt_predict)+'_'+now_time+saved_img.split()[-1]) 
            print()
            print(" "+str(idx+1) + "번째 결과")
            print(" Detect 클래스 : ", class_names[int(box.cls)])
            print(" confidence 확률 : ", box.conf)
            print(" 중심x, 중심y, 너비, 높이 : ", box.xywh)
            print()
            print("================================================================================")
            cnt_predict += 1



#### predict 실행 (try_infer/images.jpg)

In [39]:
run('capture_data/data_001.jpg')


image 1/1 c:\backend_study\META_August_PJT\ljm\capture_data\data_001.jpg: 384x640 3 banners, 59.0ms
Speed: 3.0ms preprocess, 59.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict2


================================= Predict 결과 =================================



FileNotFoundError: [Errno 2] No such file or directory: ''

In [38]:
run('capture_data/data_002.jpg')


image 1/1 c:\backend_study\META_August_PJT\ljm\capture_data\data_002.jpg: 416x640 3 banners, 1 frame, 22.1ms
Speed: 3.0ms preprocess, 22.1ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict2


================================= Predict 결과 =================================



FileNotFoundError: [Errno 2] No such file or directory: ''